<a href="https://colab.research.google.com/github/XueyanZhang/MachineLearningCompilation/blob/master/MLC_Automatic_Program_Opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic Program Optimization

MLC process can be viewed as transformation among tensor functions.

There are many ways to transform.

Which transformation is better?

In [1]:
!python3 -m  pip install mlc-ai-nightly -f https://mlc.ai/wheels

import numpy as np
import tvm
from tvm import relax
from tvm.ir.module import IRModule
from tvm.script import relax as R
from tvm.script import tir as T

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://mlc.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 MB 12.1 MB/s eta 0:00:00


# Recap: Transform a Primitive Tensor Func

In [2]:
f32 = "float32"
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def main(
        A: T.Buffer((128, 128), f32),
        B: T.Buffer((128, 128), f32),
        C: T.Buffer((128, 128), f32),
    ):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i, j, k in T.grid(128, 128, 128):
            with T.block("C"):
                vi, vj, vk = T.axis.remap("SSR", [i, j ,k])
                with T.init():
                    C[vi, vj] = T.float32(0)
                C[vi, vj] = C[vi, vj] + A[vi, vk] * B[vk, vj]

In [3]:
# define input values and baseline
a_np = np.random.rand(128, 128).astype(f32)
b_np = np.random.rand(128, 128).astype(f32)
c_np = a_np @ b_np

In [4]:
# run MyModule
a_tvm = tvm.nd.array(a_np)
b_tvm = tvm.nd.array(b_np)
c_tvm = tvm.nd.empty((128, 128), dtype=f32)

lib_rt = tvm.build(MyModule, target="llvm")
f_timer_before = lib_rt.time_evaluator("main", tvm.cpu())
print("Time of MyModule: ", f_timer_before(a_tvm, b_tvm, c_tvm).mean * 1000, " ms")
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

Time of MyModule:  4.4628361  ms


# Transformation: loop reordering

Let's add some simple transformations (split j as input arg `jfactor`)

(we did this before)

In [5]:
def schdule_mm(sch: tvm.tir.Schedule, jfactor=4):
    block_C = sch.get_block("C", "main")
    i, j, k = sch.get_loops(block_C)
    j0, j1 = sch.split(j, factors=[None, jfactor])
    sch.reorder(i, j0, k, j1)
    sch.decompose_reduction(block_C, k)
    return sch

In [6]:
# apply transformation
sch = tvm.tir.Schedule(MyModule)
sch = schdule_mm(sch)
sch.mod.show()

/usr/local/lib/python3.10/dist-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [7]:
# run the mod
lib_rt_mod = tvm.build(sch.mod, target="llvm")
f_timer_after = lib_rt_mod.time_evaluator("main", tvm.cpu())
print("Time of MyModule.mod: ", f_timer_before(a_tvm, b_tvm, c_tvm).mean * 1000, " ms")
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

Time of MyModule.mod:  4.7245186  ms


`sch.mod` should take less time theoretically. It may be subjected to noises.

# Transformation Trace

`tir.Schedule` offers a trace field, showing the steps to get a transformed module.

In [8]:
print(sch.trace) # exact transformation in schdule_mm

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  l4, l5 = sch.split(loop=l2, factors=[None, 4], preserve_unit_iters=True)
  sch.reorder(l1, l4, l3, l5)
  b6 = sch.decompose_reduction(block=b0, loop=l3)


# Stochastic Transformation

use stochastic elements in tranformation function to see which achieves better performance.

In [9]:
def stochastic_schdule_mm(sch: tvm.tir.Schedule): # no longer specify jfactors
    block_C = sch.get_block("C", "main")
    i, j, k = sch.get_loops(block_C)
    j_factors = sch.sample_perfect_tile(loop=j, n=2) # stochastic
    j0, j1 = sch.split(j, factors=j_factors)
    sch.reorder(i, j0, k, j1)
    sch.decompose_reduction(block_C, k)
    return sch

possible j_factors: [8, 16], [32, 4], [2, 64], [1,128]

In [10]:
sch = tvm.tir.Schedule(MyModule)
sch = stochastic_schdule_mm(sch)
sch.mod.show()

/usr/local/lib/python3.10/dist-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [11]:
# check the trace to see the jfactors
print(sch.trace)

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[64, 2])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l1, l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)


# Search over Stochastic Transformations

with a stochastic transformation function, there is a search space of programs (a set of possible variant programs).

with a search space, what is the best choice?

- it could be machine dependant. different machines favor different variant programs.

Let's try the simplest algorithm: **random search**

In [12]:
def random_search(mod: tvm.IRModule, num_trials=5):
    best_result = None
    best_sch = None

    for i in range(num_trials):
        sch = stochastic_schdule_mm(tvm.tir.Schedule(mod))
        lib = tvm.build(sch.mod, target='llvm')
        f_timer_after = lib.time_evaluator('main', tvm.cpu())
        result = f_timer_after(a_tvm, b_tvm, c_tvm).mean

        print(f"====== trial {i}, time {result * 1000} ms")
        print(sch.trace)

        if best_result is None or result < best_result:
            best_result = result
            best_sch = sch

    return best_sch

In [13]:
sch = random_search(MyModule)

====== trial 0, time 3.4508519000000004 ms
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[64, 2])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l1, l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)
====== trial 1, time 1.6079872000000002 ms
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[16, 8])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l1, l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)
====== trial 2, time 1.7800772 ms
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get

In [14]:
print(sch.trace)

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[8, 16])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l1, l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)


## meta_schedule

tvm offers `tune_tir` API to search for an optimizied solution within the search space specified by a stochastic transformation function

In [16]:
from tvm import meta_schedule as ms

database = ms.tune_tir(
    mod=MyModule,
    target="llvm --num-cores=1",
    max_trials_global=64,
    num_trials_per_iter=64,
    space=ms.space_generator.ScheduleFn(stochastic_schdule_mm),
    work_dir="./turn_tmp",
    task_name="main"
)

sch = ms.tir_integration.compile_tir(database, MyModule, "llvm --num-cores=1")

2023-05-04 11:39:25 [INFO] Logging directory: ./turn_tmp/logs
2023-05-04 11:39:50 [INFO] LocalBuilder: max_workers = 1
2023-05-04 11:39:53 [INFO] LocalRunner: max_workers = 1
2023-05-04 11:39:55 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"
2023-05-04 11:39:55 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2023-05-04 11:39:56 [INFO] [task_scheduler.cc:193] Sending 5 sample(s) to builder
2023-05-04 11:39:59 [INFO] [task_scheduler.cc:195] Sending 5 sample(s) to runner
2023-05-04 11:40:00 [DEBUG] XGB iter   0: tr-p-rmse: 0.241205	tr-a-peak@32: 1.000000	tr-rmse: 0.281914	tr-rmse: 0.281914
2023-05-04 11:40:00 [DEBUG] XGB iter  25: tr-p-rmse: 0.036077	tr-a-peak@32: 1.000000	tr-rmse: 0.037404	tr-rmse: 0.037404
2023-05-04 11:40:00 [DEBUG] XGB iter  50: tr-p-rmse: 0.037348	tr-a-peak@32: 1.000000	tr-rmse: 0.036806	tr-rmse: 0.036806
2023-05-04 11:40:00 [DEBUG] XGB stopped. Best iteration: [22] tr-p-rmse:0.03588	tr-a-peak@32:1.00000	tr-rmse:0.03842	tr-rmse:0.03842

In [23]:
lib = tvm.build(sch.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule after tuning: %.3f ms" % (f_timer_after(a_tvm, b_tvm, c_tvm).mean * 1000))

Time cost of MyModule after tuning: 0.375 ms


In [17]:
sch.trace.show()

/usr/local/lib/python3.10/dist-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [18]:
sch.mod.show()

/usr/local/lib/python3.10/dist-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


### Summary

In both search, split factor of [8, 16] offers the best speedup. (leveraging cache)

# Auto Scheduling

Last section, we specified the search space to be
`space=ms.space_generator.ScheduleFn(stochastic_schdule_mm)`.

`meta_schedule` comes with its own built-in set of generic stochastic transformations that work for a wide range of TensorIRs.

In [19]:
database = ms.tune_tir(
    mod=MyModule,
    target="llvm --num-cores=1",
    max_trials_global=64,
    num_trials_per_iter=64,
    # space=ms.space_generator.ScheduleFn(stochastic_schdule_mm),
    work_dir="./turn_tmp",
    task_name="main"
)

sch = ms.tir_integration.compile_tir(database, MyModule, "llvm --num-cores=1")

2023-05-04 11:50:25 [INFO] Logging directory: ./turn_tmp/logs
2023-05-04 11:50:25 [INFO] LocalBuilder: max_workers = 1
2023-05-04 11:50:26 [INFO] LocalRunner: max_workers = 1
2023-05-04 11:50:28 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"
2023-05-04 11:50:28 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2023-05-04 11:50:39 [INFO] [task_scheduler.cc:193] Sending 64 sample(s) to builder
2023-05-04 11:52:27 [INFO] [task_scheduler.cc:195] Sending 64 sample(s) to runner
2023-05-04 11:52:44 [DEBUG] XGB iter   0: tr-p-rmse: 0.469479	tr-a-peak@32: 1.000000	tr-rmse: 0.258036	tr-rmse: 0.258036
2023-05-04 11:52:44 [DEBUG] XGB iter  25: tr-p-rmse: 0.048208	tr-a-peak@32: 0.999594	tr-rmse: 0.298526	tr-rmse: 0.298526
2023-05-04 11:52:44 [DEBUG] XGB iter  50: tr-p-rmse: 0.047940	tr-a-peak@32: 0.999594	tr-rmse: 0.298858	tr-rmse: 0.298858
2023-05-04 11:52:44 [DEBUG] XGB iter  75: tr-p-rmse: 0.047940	tr-a-peak@32: 0.999594	tr-rmse: 0.298858	tr-rmse: 0.298858
2023-05-0

In [25]:
# faster result after auto tuning
lib = tvm.build(sch.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule after AUTO tuning: %.3f ms" % (f_timer_after(a_tvm, b_tvm, c_tvm).mean * 1000))

Time cost of MyModule after AUTO tuning: 0.263 ms


In [26]:
sch.trace.show() # much complex transformation

/usr/local/lib/python3.10/dist-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [27]:
sch.mod.show()

/usr/local/lib/python3.10/dist-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


### Summary
1. stochastic schedule form a search space of possible programs
2. meta schedule (`tune_tir`) searches within a space
3. meta schedule offers generic builtin search space